# Práctica 3 - Redes Neuronales Residuales 

### Natalia Martínez García, Lucía Vega Navarrete
### Grupo: AP.11.06

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import layers, metrics, optimizers, losses
from pathlib import Path
import os
import random

In [2]:
# Fijamos la semilla para poder reproducir los resultados
seed=1234
os.environ['PYTHONHASHSEED']=str(seed)
tf.random.set_seed(seed)
np.random.seed(seed)
random.seed(seed)

### 1. Carga y preprocesado del dataset 

#### 1.1. Carga 

In [3]:
DATA_PATH = Path("HIGGS.csv.gz") # El archivo está en la misma carpeta que el script

# Cargar el dataset 
higgs = pd.read_csv(DATA_PATH, compression="gzip", header=None)

print("Tamaño total del dataset (ejemplos, columnas):", higgs.shape)

# Separar características (X) y etiquetas (y)
X = higgs.iloc[:, 1:].to_numpy(dtype=np.float32) # Columnas 1-28: features
y = higgs.iloc[:, 0].to_numpy(dtype=np.float32) # Columna 0: label

# División como dice el enunciado
N_TRAIN = 2000000
N_TEST = 500000

X_train = X[:N_TRAIN]
y_train = y[:N_TRAIN]

X_test  = X[-N_TEST:]
y_test  = y[-N_TEST:]

X_extra = X[N_TRAIN:-N_TEST]
y_extra = y[N_TRAIN:-N_TEST]

print("División del dataset:")
print(f"Train: {X_train.shape}")
print(f"Test:  {X_test.shape}")
print(f"Extra: {X_extra.shape}")

Tamaño total del dataset (ejemplos, columnas): (11000000, 29)
División del dataset:
Train: (2000000, 28)
Test:  (500000, 28)
Extra: (8500000, 28)


#### 1.2. Preprocesado

In [4]:
# Normalizar usando solo TRAIN
# Media y desviación por columna (feature) 
# (x - media_train) / std_train

"""
Normalizamos solo en train porque cualquier información del test no puede influir en el entrenamiento (el modelo solo puede ver estadísticas del train).
Si usamos la media y desviación del test para normalizar, meteríamos información del futuro dentro del entrenamiento.

Eso se llama data leakage (filtración de información) y hace que tus resultados no sean realistas.
"""

mean_train = X_train.mean(axis=0) # axis = 0 para calcularlo por columnas (cada feature)
std_train  = X_train.std(axis=0)

# Evitar división por cero
std_train[std_train == 0] = 1.0

# Aplicar la transformación estándar: (x - media) / sd
X_train = (X_train - mean_train) / std_train
X_extra = (X_extra - mean_train) / std_train
X_test  = (X_test  - mean_train) / std_train

print(f"- Media global train (ya normalizado): {X_train.mean():.5f}")
print(f"- Std global train  (ya normalizado): {X_train.std():.5f}")

# Comprobación del desbalanceo de clases
def class_stats(name, y):
    n = len(y)
    n_signal = np.sum(y == 1)
    n_back   = np.sum(y == 0)
    print(f"\n{name}:")
    print(f"Signal = {n_signal} ({100*n_signal/n:.2f}%)")
    print(f"Background = {n_back}   ({100*n_back/n:.2f}%)")

print("\n Distribución de clases:")
class_stats("Train", y_train)
class_stats("Extra", y_extra)
class_stats("Test",  y_test)

- Media global train (ya normalizado): -0.00000
- Std global train  (ya normalizado): 1.00000

 Distribución de clases:

Train:
Signal = 1058818 (52.94%)
Background = 941182   (47.06%)

Extra:
Signal = 4505798 (53.01%)
Background = 3994202   (46.99%)

Test:
Signal = 264507 (52.90%)
Background = 235493   (47.10%)


### 2. Creación de red neuronal con capas residuales

In [5]:
class ResidualBlock(tf.keras.models.Model):
    
    def __init__(self, units=128, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        # Inicializamos las capas en __init__ en lugar de en build
        self.dense1 = layers.Dense(units, activation='relu', name=f"{self.name}_dense1")
        self.dense2 = layers.Dense(units, activation=None, name=f"{self.name}_dense2")
        self.add = layers.Add(name=f"{self.name}_add")
        self.activation = layers.Activation('relu')

    def call(self, x):
        """Forward pass con conexión residual."""
        x_skip = x
        out = self.dense1(x)
        out = self.dense2(out)
        out = self.add([out, x_skip])
        out = self.activation(out)
        return out
    
    def get_config(self):
        config = super().get_config()
        config.update({'units': self.units})
        return config

In [6]:
class ResidualNetwork(tf.keras.Model):
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        # Inicializamos todas las capas en __init__
        self.capa_entrada = layers.Dense(128, activation="relu", name="capa_entrada")
        self.res_block1 = ResidualBlock(units=128, name="res_block1")
        self.res_block2 = ResidualBlock(units=128, name="res_block2")
        self.capa_intermedia = layers.Dense(64, activation="relu", name="capa_intermedia")
        self.capa_salida = layers.Dense(1, activation="sigmoid", name="capa_salida")
    
    def call(self, inputs):
        """Forward pass de la red completa."""
        x = self.capa_entrada(inputs)
        x = self.res_block1(x)
        x = self.res_block2(x)
        x = self.capa_intermedia(x)
        outputs = self.capa_salida(x)
        return outputs
    
    def get_config(self):
        return super().get_config()

In [7]:
# CONSTRUIR EL MODELO
modelo_base = ResidualNetwork(name="red_residual")
_ = modelo_base(X_train[:1]) # # Para que podamos ver el summary
modelo_base.summary()

Model: "red_residual"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ capa_entrada (Dense)            │ (1, 128)               │         3,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_block1 (ResidualBlock)      │ ?                      │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_block2 (ResidualBlock)      │ ?                      │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ capa_intermedia (Dense)         │ (1, 64)                │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ capa_salida (Dense)             │ (1, 1)                 │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 78,081 (305.00 KB)

 Trainable params: 78,081 (305.00 KB)

 Non-trainable params: 0 (0.00 B)

### 3. Entrenamiento de la red

En esta práctica hemos implementado el entrenamiento de la red siguiendo la estructura del tutorial “Escribir un ciclo de entrenamiento desde cero” de TensorFlow, proporcionado en el enunciado de esta práctica.
El código hace exactamente los mismos bloques lógicos propuestos en el tutorial, lo único diferente es que o ampliamos un poco para el cálculo de las métricas F1Score y balanced accurcay. 

In [8]:
BATCH_SIZE = 4096

# Convierte arrays de numpy en un objeto tf.data.Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
# shuffle: Mezcla aleatoriamente los datos
# batch: Divide los datos en batches
train_dataset = train_dataset.shuffle(buffer_size=100_000, reshuffle_each_iteration=True).batch(BATCH_SIZE)

# Dataset de test: solo en batches, sin shuffle
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_dataset = test_dataset.batch(BATCH_SIZE)

print(train_dataset)
print(test_dataset)

<_BatchDataset element_spec=(TensorSpec(shape=(None, 28), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.float32, name=None))>
<_BatchDataset element_spec=(TensorSpec(shape=(None, 28), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.float32, name=None))>


2. Definir loss, optimizador y métricas

In [9]:
optimizer = optimizers.Adam(learning_rate=1e-3) # Optimizador
loss_fn = losses.BinaryCrossentropy() # Función de loss (binaria, porque la salida es 0/1)

# Métricas para TRAIN
train_loss = metrics.Mean(name="train_loss")
train_accuracy = metrics.BinaryAccuracy(name="train_accuracy")

# Métricas para TEST
test_loss = metrics.Mean(name="test_loss") # ELIMINARLO ? NO LO PIDE Y NO LO USAMOS
test_accuracy = metrics.BinaryAccuracy(name="test_accuracy")
# Toca calcular el F1 a mano tb xq me estaba dando un error de dimensionalidad por usar la clase q lo hace directo de keras lmao 
test_precision = metrics.Precision(name="test_precision")
test_recall    = metrics.Recall(name="test_recall")

In [10]:
@tf.function
def train_step(x, y):
    """Paso de entrenamiento: forward + loss + backprop + métricas train."""
    with tf.GradientTape() as tape:
        predictions = modelo_base(x, training=True)
        loss_value = loss_fn(y, predictions)
    grads = tape.gradient(loss_value, modelo_base.trainable_weights)
    optimizer.apply_gradients(zip(grads, modelo_base.trainable_weights))

    # Actualizar métricas de entrenamiento
    train_loss.update_state(loss_value)
    train_accuracy.update_state(y, predictions)
    return loss_value

@tf.function
def test_step(x, y):
    """
    Paso de evaluación:
      - forward
      - actualizar métricas de test (loss, accuracy, precision, recall)
    Devuelve las predicciones para poder calcular balanced accuracy fuera.
    """
    predictions = modelo_base(x, training=False)
    loss_value = loss_fn(y, predictions)

    test_loss.update_state(loss_value)
    test_accuracy.update_state(y, predictions)
    test_precision.update_state(y, predictions)
    test_recall.update_state(y, predictions)

    return predictions

In [11]:
import time

def entrenar_modelo(train_dataset, train_step, train_loss, train_accuracy, epochs=3):
    # LE PASAMOS PARAMETROS PORQUE PARA CADA MODELO DISTINTO HAY QUE DEFINIR UN TRAIN_STEP DISTINTO
    # ASI QUE LO DEFINIMOS FUERA Y SE LO PASAMOS COMO PARAMETRO
    for epoch in range(epochs):
        print("\nStart of epoch %d" % (epoch,))
        start_time = time.time()

        # Iterar sobre los batches del dataset 
        for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
            loss_value = train_step(x_batch_train, y_batch_train)

            # Log cada N batches 
            if step % 200 == 0:
                print(
                    "Training loss (for one batch) at step %d: %.4f"
                    % (step, float(loss_value))
                )
                print("Seen so far: %d samples" % ((step + 1) * BATCH_SIZE))

        # Mostrar las métricas al final de cada epoch 
        train_acc  = float(train_accuracy.result())
        train_loss_epoch = float(train_loss.result())
        print("\n--- Training metrics ---")
        print("Accuracy: %.4f" % train_acc)
        print("Loss: %.4f" % train_loss_epoch)

        # Resetear las métricas al final del epoch
        train_loss.reset_state()
        train_accuracy.reset_state()
        
        print("Time taken: %.2fs" % (time.time() - start_time))

In [12]:
entrenar_modelo(train_dataset, train_step, train_loss, train_accuracy, epochs=10)


Start of epoch 0
Training loss (for one batch) at step 0: 0.7083
Seen so far: 4096 samples
Training loss (for one batch) at step 200: 0.5655
Seen so far: 823296 samples
Training loss (for one batch) at step 400: 0.5309
Seen so far: 1642496 samples

--- Training metrics ---
Accuracy: 0.7004
Loss: 0.5679
Time taken: 6.87s

Start of epoch 1
Training loss (for one batch) at step 0: 0.5289
Seen so far: 4096 samples
Training loss (for one batch) at step 200: 0.5310
Seen so far: 823296 samples
Training loss (for one batch) at step 400: 0.5284
Seen so far: 1642496 samples

--- Training metrics ---
Accuracy: 0.7348
Loss: 0.5236
Time taken: 6.54s

Start of epoch 2
Training loss (for one batch) at step 0: 0.5009
Seen so far: 4096 samples
Training loss (for one batch) at step 200: 0.5144
Seen so far: 823296 samples
Training loss (for one batch) at step 400: 0.5047
Seen so far: 1642496 samples

--- Training metrics ---
Accuracy: 0.7440
Loss: 0.5091
Time taken: 6.59s

Start of epoch 3
Training loss

In [13]:
def evaluar_modelo(test_dataset, test_step, test_accuracy, test_precision, test_recall):
    # Variables para calcular Balanced Accuracy
    y_true_all = []  # Todas las etiquetas reales
    y_pred_all = []  # Todas las predicciones binarias
    
    # Iterar sobre el dataset de test
    for x_batch_test, y_batch_test in test_dataset:
        # test_step actualiza loss, accuracy, precision, recall
        test_predictions = test_step(x_batch_test, y_batch_test)
        
        # Guardar para balanced accuracy
        y_true_all.append(y_batch_test.numpy().astype(np.int32))
        y_pred_all.append((test_predictions.numpy() >= 0.5).astype(np.int32))
    
    y_true_all = np.concatenate(y_true_all, axis=0).reshape(-1)
    y_pred_all = np.concatenate(y_pred_all, axis=0).reshape(-1)
    
    # Métricas de test a partir de los objetos de Keras
    test_acc = float(test_accuracy.result())
    precision = float(test_precision.result())
    recall = float(test_recall.result())
    f1_score = 2 * precision * recall / (precision + recall + 1e-8)
    
    # Balanced Accuracy (a mano)
    mask_0 = (y_true_all == 0)  # pone a True todos los valores de la lista de etiquetas reales que valgan 0
    mask_1 = (y_true_all == 1)  # pone a True todos los valores de la lista de etiquetas reales que valgan 1
    
    acc_class0 = np.mean(y_pred_all[mask_0] == y_true_all[mask_0])
    acc_class1 = np.mean(y_pred_all[mask_1] == y_true_all[mask_1])
    balanced_accuracy = (acc_class0 + acc_class1) / 2
    
    # Crear diccionario con resultados
    # Lo guardamos porque luego hay que compararlo con el que lleva fine tuning !!!
    resultados = {
        'accuracy': test_acc,
        'balanced_accuracy': balanced_accuracy,
        'f1_score': f1_score,
        'precision': precision,
        'recall': recall,
        'sensitivity': acc_class1,
        'specificity': acc_class0
    }
    
    return resultados

def mostrar_resultados_test(resultados):
    print("\n--- Test metrics ---")
    print("Accuracy: %.4f" % resultados['accuracy'])
    print("Precision: %.4f" % resultados['precision'])
    print("Recall: %.4f" % resultados['recall'])
    print("F1-Score: %.4f" % resultados['f1_score'])
    
    print("\n--- Balanced Accuracy ---")
    print("Class 0 (background): %.4f" % resultados['specificity'])
    print("Class 1 (signal): %.4f" % resultados['sensitivity'])
    print("Media: %.4f" % resultados['balanced_accuracy'])

In [14]:
# Evaluar en test
resultados_test = evaluar_modelo(test_dataset,test_step,test_accuracy,test_precision,test_recall)
mostrar_resultados_test(resultados_test)


--- Test metrics ---
Accuracy: 0.7597
Precision: 0.7637
Recall: 0.7903
F1-Score: 0.7768

--- Balanced Accuracy ---
Class 0 (background): 0.7254
Class 1 (signal): 0.7903
Media: 0.7579


In [15]:
# Guardar el modelo entrenado
MODEL_PATH = "residual_network_entrenada.keras"
modelo_base.save(MODEL_PATH)
print(f"\nModelo guardado en: {MODEL_PATH}")


Modelo guardado en: residual_network_entrenada.keras


### 4. Fine tuning

#### 4.1 Cargar el modelo base ya entrenado

In [56]:
# Cargar el modelo base entrenado
MODEL_PATH = "residual_network_entrenada.keras"
# LE PONEMOS OTRO NOMBRE A LA VARIABLE O DEJAMOS LA MISMA ????
modelo_finetuning = keras.models.load_model(
    MODEL_PATH,
    custom_objects={'ResidualBlock': ResidualBlock, 'ResidualNetwork': ResidualNetwork}
)
print("Modelo cargado desde:", MODEL_PATH)
modelo_finetuning.summary()

Modelo cargado desde: residual_network_entrenada.keras


Model: "red_residual"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ capa_entrada (Dense)            │ (1, 128)               │         3,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_block1 (ResidualBlock)      │ ?                      │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_block2 (ResidualBlock)      │ ?                      │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ capa_intermedia (Dense)         │ (1, 64)                │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ capa_salida (Dense)             │ (1, 1)                 │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 78,081 (305.00 KB)

 Trainable params: 78,081 (305.00 KB)

 Non-trainable params: 0 (0.00 B)

#### 4.2 Congelar los pesos

In [57]:
# Congelar todas las capas del modelo
for layer in modelo_finetuning.layers:
    layer.trainable = False

# Comprobamos que están congeladas
print("Capas del modelo y su estado de entrenamiento:")
for layer in modelo_finetuning.layers:
    print(f"  {layer.name}: trainable={layer.trainable}")

modelo_finetuning.summary()

Capas del modelo y su estado de entrenamiento:
  capa_entrada: trainable=False
  res_block1: trainable=False
  res_block2: trainable=False
  capa_intermedia: trainable=False
  capa_salida: trainable=False


Model: "red_residual"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ capa_entrada (Dense)            │ (1, 128)               │         3,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_block1 (ResidualBlock)      │ ?                      │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_block2 (ResidualBlock)      │ ?                      │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ capa_intermedia (Dense)         │ (1, 64)                │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ capa_salida (Dense)             │ (1, 1)                 │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 78,081 (305.00 KB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 78,081 (305.00 KB)

#### 4.3 Añadir módulos de adaptación de baja dimensión

In [58]:
class CapaModeloFineTuning(tf.keras.layers.Layer):
    """
    Capa Dense con adaptación de baja dimensión.
    y = (W + α*B*A)*x + b
    donde W está congelado y solo A y B son entrenables.
    """
    
    def __init__(self, capa_original, r=8, alpha=0.1, **kwargs):
        super().__init__(**kwargs)
        self.capa_original = capa_original
        self.r = r  # Rango de la descomposición
        self.alpha = alpha  # Factor de escala
    
        # Obtener dimensiones de la capa original
        W = capa_original.weights[0]  # Matriz de pesos
        # CORRECCIÓN: W en Keras Dense tiene shape (din, dout), NO (dout, din)
        self.din, self.dout = W.shape  # Cambiado el orden!
        
        # Inicializar matrices A y B
        # A: (r, din) - inicializada aleatoriamente
        self.A = self.add_weight(
            name=f"{self.name}_A",
            shape=(self.r, self.din),
            initializer='glorot_uniform', # MIRAR SI ESTA ES INICIALIZACION ALEATORIA DE VERDAD
            trainable=True
        )
        
        # B: (dout, r) - inicializada a cero
        self.B = self.add_weight(
            name=f"{self.name}_B",
            shape=(self.dout, self.r),
            initializer='zeros',
            trainable=True
        )
    
    def call(self, x):
        # Salida original con W congelado
        y_original = self.capa_original(x)
        
        # Calcular BA (adaptación de baja dimensión)
        # B: (dout, r), A: (r, din) -> BA: (dout, din)
        BA = tf.matmul(self.B, self.A)
        
        # Aplicar BA a x: (dout, din) * (batch, din)^T = (batch, dout)
        lora_output = self.alpha * tf.matmul(x, BA, transpose_b=True)
        
        return y_original + lora_output
            
    def get_config(self):
        config = super().get_config()
        config.update({
            'r': self.r,
            'alpha': self.alpha
        })
        return config

In [59]:
class ResidualBlockFineTuning(tf.keras.models.Model):
    
    def __init__(self, original_block, r=4, alpha=0.1, **kwargs):
        super().__init__(**kwargs)
        self.original_block = original_block
        self.r = r
        self.alpha = alpha
        
        # Envolver las capas Dense del bloque original con la CapaModeloFineTuning
        self.dense1 = CapaModeloFineTuning(original_block.dense1, r=r, alpha=alpha, name=f"{self.name}_dense1_ft")
        self.dense2 = CapaModeloFineTuning(original_block.dense2, r=r, alpha=alpha, name=f"{self.name}_dense2_ft")
        self.add = original_block.add
        self.activation = original_block.activation

    def call(self, x):
        """Forward pass con conexión residual."""
        x_skip = x
        out = self.dense1(x)
        out = self.dense2(out)
        out = self.add([out, x_skip])
        out = self.activation(out)
        return out
    
    def get_config(self):
        config = super().get_config()
        config.update({'r': self.r, 'alpha': self.alpha})
        return config

In [60]:
class ResidualNetworkFineTuning(tf.keras.Model):
    """Red residual completa con adaptación de baja dimensión"""
    
    def __init__(self, modelo_base, r=4, alpha=0.1, **kwargs):
        super().__init__(**kwargs)
        self.r = r
        self.alpha = alpha
        
        # Envolver todas las capas Dense con adaptación de baja dimensión
        self.capa_entrada = CapaModeloFineTuning(modelo_base.capa_entrada, r=r, alpha=alpha,name="ft_capa_entrada")
        self.res_block1 = ResidualBlockFineTuning(modelo_base.res_block1, r=r, alpha=alpha, name="ft_res_block1")
        self.res_block2 = ResidualBlockFineTuning(modelo_base.res_block2, r=r, alpha=alpha, name="ft_res_block2")
        self.capa_intermedia = CapaModeloFineTuning(modelo_base.capa_intermedia, r=r, alpha=alpha, name="ft_capa_intermedia")
        self.capa_salida = CapaModeloFineTuning(modelo_base.capa_salida, r=r, alpha=alpha, name="ft_capa_salida")
    
    def call(self, inputs):
        x = self.capa_entrada(inputs)
        x = self.res_block1(x)
        x = self.res_block2(x)
        x = self.capa_intermedia(x)
        outputs = self.capa_salida(x)
        return outputs
    
    def get_config(self):
        config = super().get_config()
        config.update({'r': self.r, 'alpha': self.alpha})
        return config

In [69]:
r = 16  # en lugar de 8
alpha = 0.2  # en lugar de 0.1

modelo_ft = ResidualNetworkFineTuning(modelo_finetuning, r=r, alpha=alpha, name="red_residual_fine_tuning")

# Build del modelo
_ = modelo_ft(X_train[:1])
modelo_ft.summary()

print(f"Rango r: {r}")
print(f"Factor alpha: {alpha}")

Model: "red_residual_fine_tuning"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ ft_capa_entrada                 │ ?                      │         6,208 │
│ (CapaModeloFineTuning)          │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ft_res_block1                   │ ?                      │        41,216 │
│ (ResidualBlockFineTuning)       │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ft_res_block2                   │ ?                      │        41,216 │
│ (ResidualBlockFineTuning)       │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ft_capa_intermedia              │ ?                      │        11,328 │
│ (CapaModeloFineTuning)          │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ft_capa_salida                  │ ?                      │         1,105 │
│ (CapaModeloFineTuning)          │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 101,073 (394.82 KB)

 Trainable params: 22,992 (89.81 KB)

 Non-trainable params: 78,081 (305.00 KB)

Rango r: 16
Factor alpha: 0.2


### 4.4 Entrenar el modelo adaptado

In [70]:
# Preparar dataset extra
extra_dataset = tf.data.Dataset.from_tensor_slices((X_extra, y_extra))
extra_dataset = extra_dataset.shuffle(buffer_size=100_000, reshuffle_each_iteration=True).batch(BATCH_SIZE)

In [71]:
# Definir nuevas métricas y optimizador para fine tuning

ft_optimizer = optimizers.Adam(learning_rate=5e-4) # PONER UN LEARNING RATE MENOR
loss_fn_ft = losses.BinaryCrossentropy()

# Crear nuevas métricas para el fine-tuning
ft_train_loss = metrics.Mean(name="ft_train_loss")
ft_train_accuracy = metrics.BinaryAccuracy(name="ft_train_accuracy")

ft_test_accuracy = metrics.BinaryAccuracy(name="ft_test_accuracy")
ft_test_precision = metrics.Precision(name="ft_test_precision")
ft_test_recall = metrics.Recall(name="ft_test_recall")

In [72]:
@tf.function
def ft_train_step(x, y):
    """Paso de entrenamiento para fine-tuning"""
    with tf.GradientTape() as tape:
        predictions = modelo_ft(x, training=True)
        loss_value = loss_fn_ft(y, predictions)
    
    # Solo calcular gradientes para variables entrenables (A y B)
    grads = tape.gradient(loss_value, modelo_ft.trainable_variables)
    ft_optimizer.apply_gradients(zip(grads, modelo_ft.trainable_variables))
    
    ft_train_loss.update_state(loss_value)
    ft_train_accuracy.update_state(y, predictions)
    return loss_value

@tf.function
def ft_test_step(x, y):
    """Paso de evaluación para fine-tuning"""
    predictions = modelo_ft(x, training=False)
    
    ft_test_accuracy.update_state(y, predictions)
    ft_test_precision.update_state(y, predictions)
    ft_test_recall.update_state(y, predictions)
    
    return predictions

In [73]:
entrenar_modelo(extra_dataset, ft_train_step, ft_train_loss, ft_train_accuracy, epochs=3)


Start of epoch 0
Training loss (for one batch) at step 0: 0.4903
Seen so far: 4096 samples
Training loss (for one batch) at step 200: 0.4793
Seen so far: 823296 samples
Training loss (for one batch) at step 400: 0.4750
Seen so far: 1642496 samples
Training loss (for one batch) at step 600: 0.4952
Seen so far: 2461696 samples
Training loss (for one batch) at step 800: 0.4929
Seen so far: 3280896 samples
Training loss (for one batch) at step 1000: 0.4706
Seen so far: 4100096 samples
Training loss (for one batch) at step 1200: 0.4773
Seen so far: 4919296 samples
Training loss (for one batch) at step 1400: 0.4815
Seen so far: 5738496 samples
Training loss (for one batch) at step 1600: 0.4790
Seen so far: 6557696 samples
Training loss (for one batch) at step 1800: 0.4723
Seen so far: 7376896 samples
Training loss (for one batch) at step 2000: 0.4788
Seen so far: 8196096 samples

--- Training metrics ---
Accuracy: 0.7616
Loss: 0.4826
Time taken: 41.40s

Start of epoch 1
Training loss (for o

### 4.5. Evaluar en test y comparar los resultados

In [74]:
resultados_test_ft = evaluar_modelo(
    test_dataset, 
    ft_test_step, 
    test_accuracy, 
    test_precision, 
    test_recall
)
mostrar_resultados_test(resultados_test_ft)


--- Test metrics ---
Accuracy: 0.7597
Precision: 0.7637
Recall: 0.7903
F1-Score: 0.7768

--- Balanced Accuracy ---
Class 0 (background): 0.7365
Class 1 (signal): 0.7864
Media: 0.7615


In [75]:
# Comparación
print("\n" + "="*60)
print("COMPARACIÓN DE RESULTADOS")
print("="*60)
print(f"{'Métrica':<25} {'Modelo Base':<15} {'Modelo LoRA':<15} {'Diferencia':<15}")
print("-"*60)
for metric in ['accuracy', 'balanced_accuracy', 'f1_score', 'precision', 'recall']:
    base_val = resultados_test[metric]
    lora_val = resultados_test_ft[metric]
    diff = lora_val - base_val
    print(f"{metric:<25} {base_val:>14.4f} {lora_val:>14.4f} {diff:>+14.4f}")


COMPARACIÓN DE RESULTADOS
Métrica                   Modelo Base     Modelo LoRA     Diferencia     
------------------------------------------------------------
accuracy                          0.7597         0.7597        +0.0000
balanced_accuracy                 0.7579         0.7615        +0.0036
f1_score                          0.7768         0.7768        +0.0000
precision                         0.7637         0.7637        +0.0000
recall                            0.7903         0.7903        +0.0000


In [76]:
# Guardar el modelo adaptado
MODEL_FT_PATH = "residual_network_lora.keras"
modelo_ft.save(MODEL_FT_PATH)
print(f"\nModelo con módulo de adaptación de baja dimensión guardado en: {MODEL_FT_PATH}")


Modelo con módulo de adaptación de baja dimensión guardado en: residual_network_lora.keras
